# Personal Coach

In [16]:
import os
import re
import base64
import mimetypes
from typing import Dict, Any, Optional
from urllib.parse import urlparse, parse_qs

In [17]:
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
from langchain.agents import create_agent
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import InMemorySaver

from tavily import TavilyClient
from youtube_transcript_api import YouTubeTranscriptApi


In [19]:

def image_file_to_data_url(image_path: str) -> str:
    """Convert a local image file into a data URL for multimodal input"""
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image file not found: {image_path}")
    
    mime_type, _ = mimetypes.guess_type(image_path)
    if mime_type is None:
        mime_type = "image/jpeg"
        
    with open(image_path, "rb") as f:
        b64 = base64.b64encode(f.read()).decode("utf-8")
    
    return f"data:{mime_type};base64,{b64}"

def extract_youtube_video_id(url: str) -> Optional[str]:
    """
    Extract YouTube video ID from common URL formats:
    - https://www.youtube.com/watch?v=VIDEO_ID
    - https://youtu.be/VIDEO_ID
    - https://www.youtube.com/shorts/VIDEO_ID
    """
    try:
        parsed = urlparse(url)

        if "youtu.be" in parsed.netloc:
            return parsed.path.strip("/")

        if "youtube.com" in parsed.netloc:
            if parsed.path == "/watch":
                return parse_qs(parsed.query).get("v", [None])[0]
            if parsed.path.startswith("/shorts/"):
                return parsed.path.split("/shorts/")[-1].split("/")[0]
            if parsed.path.startswith("/embed/"):
                return parsed.path.split("/embed/")[-1].split("/")[0]

        return None
    except Exception:
        return None


def _get_youtube_transcript_flexible(video_id: str) -> list[dict]:
    """
    Compatibility wrapper for different youtube-transcript-api versions.
    """
    # Common legacy API
    try:
        return YouTubeTranscriptApi.get_transcript(video_id, languages=["en", "es"])
    except Exception:
        pass

    # Some newer versions changed the interface
    try:
        api = YouTubeTranscriptApi()
        fetched = api.fetch(video_id, languages=["en", "es"])  # type: ignore
        # normalize if objects are returned
        normalized = []
        for item in fetched:
            if isinstance(item, dict):
                normalized.append(item)
            else:
                normalized.append(
                    {
                        "text": getattr(item, "text", ""),
                        "start": getattr(item, "start", None),
                        "duration": getattr(item, "duration", None),
                    }
                )
        return normalized
    except Exception as e:
        raise RuntimeError(f"Could not fetch transcript for video {video_id}: {e}") from e


In [29]:
# LangChain tools

tavily_client = TavilyClient()


@tool
def web_search(query: str) -> Dict[str, Any]:
    """Search the web for recipes, nutrition facts, substitutions and cooking tips"""
    try:
        results = tavily_client.search(
            query=query,
            max_results=5,
            search_depth="basic",
        )
        return results
    except Exception as e:
        return {"error": str(e), "query": query}
    
    
@tool
def get_youtube_recipe_transcript(video_url: str) -> Dict[str, Any]:
    """
    Get transcript text from a YouTube recipe/tutorial video URL.
    Useful for summarizing or adapting a video recipe.
    """
    video_id = extract_youtube_video_id(video_url)
    if not video_id:
        return {"error": "Invalid or unsupported YouTube URL", "video_url": video_url}

    try:
        transcript_items = _get_youtube_transcript_flexible(video_id)

        full_text = " ".join(
            [
                item.get("text", "").strip()
                for item in transcript_items
                if isinstance(item, dict) and item.get("text")
            ]
        )

        # Keep transcript shorter to avoid huge context windows during practice
        full_text = re.sub(r"\s+", " ", full_text).strip()
        shortened = full_text[:6000]

        return {
            "video_url": video_url,
            "video_id": video_id,
            "transcript_preview": shortened,
            "transcript_chars": len(full_text),
            "note": "Transcript may be truncated for token efficiency.",
        }
    except Exception as e:
        return {"error": str(e), "video_url": video_url, "video_id": video_id}

In [30]:
# System prompt

SYSTEM_PROMPT = """
You are a Personal Nutrition Coach and Recipe Assistant.

Your job:
- Help users cook meals with the ingredients they already have.
- Suggest healthy options based on their goals (fat loss, maintenance, muscle gain).
- When needed, use the web_search tool to find recipes, substitutions, and cooking tips.
- When the user shares a YouTube recipe video URL, use the get_youtube_recipe_transcript tool to summarize the video and adapt the recipe to the user's ingredients/goals.
- If the user provides an image of ingredients or a meal, analyze it and use that information.

Behavior:
- Ask at most 1 short clarifying question only if absolutely needed.
- Otherwise, make a helpful best effort from avaliable info.
- Provide practival recipes with:
    1) Dish name
    2) Why it fits the user's goal
    3) Ingredients (including substitutions)
    4) Steps
    5) Optional nutrition estimate (rough)
- Mention uncertainties clearly (e.g., exact calories depend on quantity).
- Do not give medical advice. For health conditions, suggest consulting a professional.

Style:
- Friendly, practical, concise, and organized.
"""

In [31]:
# Create agent

MODEL_NAME = "openai:gpt-5-nano"

agent = create_agent(
    model=MODEL_NAME,
    tools=[web_search, get_youtube_recipe_transcript],
    system_prompt=SYSTEM_PROMPT,
    checkpointer=InMemorySaver(),
)

In [34]:
# Helper functions

def ask_nutrition_coach_text(
    user_text: str,
    thread_id: str = "nutrition-thread-1",
) -> str:
    """
    Text-only interaction with memory
    """
    config = {"configurable": {"thread_id": thread_id}}
    
    response = agent.invoke(
        input={"messages":[HumanMessage(content=user_text)]},
        config=config
    )
    
    return response["messages"][-1].content
    
    
def ask_nutrition_coach_with_image(
    user_text: str,
    image_path: str,
    thread_id: str = "nutrition-thread-1"
) -> str:
    """
    Multimodal interaction (text + image)
    """    
    config = {"configurable": {"thread_id": thread_id}}
    image_data_url = image_file_to_data_url(image_path)
    
    message = HumanMessage(
        content=[
            {"type": "text", "text":user_text},
            {"type": "image_url", "image_url": {"url": image_data_url}}
        ]
    )
    
    response = agent.invoke(
        input={"messages":[message]},
        config=config,
    )
    
    return response["messages"][-1].content


def stream_nutrition_coach_text(
    user_text: str,
    thread_id: str = "nutrition-thread-1",
) -> None:
    """
    Stream agent updates (learning what's happening)
    """
    config = {"configurable": {"thread_id": thread_id}}
    
    for event in agent.stream(
        {"messages": [HumanMessage(content=user_text)]},
        config=config,
        stream_mode="updates",
    ):
        print(event)
        print("-" * 80)

In [35]:
# example

answer1 = ask_nutrition_coach_text(
    "I have leftover chicken, rice, onion and eggs. I want a high-protein dinner and I have only 20 minutes"
)

print(answer1)

Dish: Quick Chicken Fried Rice with Scrambled Eggs

Why it fits your goal: High protein from chicken and eggs, uses leftovers, and can be ready in about 20 minutes in one pan.

Ingredients (adjust to what you have)
- 1.5 cups cooked rice (about 300 g), ideally cold leftover rice
- 250–350 g cooked chicken, shredded or diced
- 1 small onion, diced
- 2 eggs
- 1–2 tbsp oil (neutral like canola/vegetable; or a small drizzle sesame oil for flavor)
- 2–3 tbsp soy sauce or tamari (to taste)
- Salt, pepper to taste
Optional
- 1 clove garlic, minced
- Green onions or any chopped herbs for garnish
- A splash of hot sauce or a pinch of chili flakes for heat
- Substitute veggies if you have them (frozen peas, carrots, bell pepper)

Steps
1) Prep ingredients quickly: dice onion, shred chicken, beat eggs in a bowl.
2) Heat a large skillet or wok over medium-high heat and add 1 tbsp oil.
3) Scramble eggs until just set, then push to one side of the pan and set aside.
4) Add the remaining oil to the p

In [36]:
answer2 = ask_nutrition_coach_text(
    "Remember that I don't eat tuna and I prefer low-oil meals."
)

print(answer2)

Dish: Low-Oil High-Protein Chicken Fried Rice with Scrambled Eggs

Why it fits your goal: Uses leftover chicken and eggs for a high-protein meal, ready in about 20 minutes, and prepared with minimal oil.

Ingredients (adjust to what you have)
- 1.5 cups cooked rice (about 300 g), ideally cold leftover rice
- 250–350 g cooked chicken, shredded or diced
- 1 small onion, diced
- 2 eggs
- 1 tsp oil (or use a light spray; nonstick pan helps)
- 1–2 tbsp soy sauce or tamari (to taste)
- Salt and pepper to taste
Optional (if you have them)
- 1 clove garlic, minced
- Green onions or any chopped herbs for garnish
- A splash of hot sauce or chili flakes
- Extra veggies (frozen peas, carrots, bell pepper)

Steps
1) Prep: Dice the onion, shred/dice chicken, and beat the eggs in a small bowl.
2) Heat a nonstick skillet over medium-high heat and add 1 tsp oil (or a light spray). If using garlic, add it now and sauté 30 seconds.
3) Add onion and sauté 1–2 minutes until translucent. If using a splash o

In [37]:
answer3 = ask_nutrition_coach_text(
    "Now suggest 2 breakfast ideas with bread, eggs and banana."
)

print(answer3)

Here are two quick breakfast ideas using bread, eggs, and banana.

1) Banana-Protein Egg French Toast
- Why it fits: High protein from eggs, natural sweetness from banana, quick to make in about 15–20 minutes. Uses minimal oil if you’re cautious.
- Ingredients (adjust as you have):
  - 2 slices bread (preferably whole grain)
  - 2 eggs
  - 1 small banana, mashed
  - 1 tsp oil or nonstick spray
  - Optional: 1/2 tsp cinnamon, a splash of vanilla, a pinch of salt, maple syrup or honey for serving
- Steps:
  1) Mash banana in a bowl, whisk in eggs (and cinnamon/vanilla if using).
  2) Heat a nonstick skillet with 1 tsp oil or spray.
  3) Dip each bread slice in the egg-banana mixture, allowing brief soak, then cook 2–3 minutes per side until golden.
  4) Serve with extra banana slices on top and a light drizzle of syrup if you like.
- Rough nutrition (approx): 25–40 g protein total depending on bread and egg size; ~350–500 kcal per serving.

2) Banana-Egg Breakfast Sandwich
- Why it fits:

In [38]:
answer4 = ask_nutrition_coach_text(
    "I found this recipe videl: https://www.youtube.com/watch?v=8gZrPB1RJHo"
    "Can you summarize it and adapt it to a fat-loss dinner with chicken and vegetables?"
)

print(answer4)

Here’s a compact, fat-loss friendly adaptation of the video you shared, focused on chicken and vegetables.

Dish: Lean Herbed Chicken and Veggie Broth (Fat-Loss Dinner)

Why it fits fat loss: 
- High protein from chicken helps satiety and muscle maintenance.
- Lots of vegetables add volume with few calories.
- Uses mostly water/broth (minimal oil) for flavor, with a light starch option if desired.

Ingredients (adjust to what you have)
- 400–500 g skinless chicken breast or thighs, cut into bite-sized pieces
- 1 medium onion, chopped
- 2 garlic cloves, minced
- 1 green bell pepper, chopped
- 2 carrots, sliced
- 1 small zucchini or broccoli florets (optional)
- 4–6 cups water or low-sodium chicken broth
- 1–2 sprigs cilantro or a small handful cilantro stems
- Salt and pepper to taste
- Optional for flavor: 1 bay leaf, pinch chili flakes, a squeeze of lemon, or 1 tsp cumin/paprika
- Optional serving add-ins (for texture if you want a small carb): 1/2 cup cooked brown rice or cauliflower

In [41]:
stream_nutrition_coach_text(
    "Search the web for 3 healthy lentil and rice recipes and summarize them."
)

{'model': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 586, 'prompt_tokens': 4648, 'total_tokens': 5234, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 512, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 3840}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-DENKy7Nqo9t2219snTJUrTtagcaPa', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019ca66c-c439-7c30-b166-e466cc600f2b-0', tool_calls=[{'name': 'web_search', 'args': {'query': 'healthy lentil rice recipe one-pot'}, 'id': 'call_U5hB9LItCMt54G6hmtMSG8To', 'type': 'tool_call'}, {'name': 'web_search', 'args': {'parameters': {'query': 'lentil and rice dish healthy'}, '}} ,{': ':'}, 'id': 'call_hU1GHXivC92XOwRjUzddqky1', 'type': 'tool_call'}], invalid_

In [42]:
from pprint import pprint
from langchain_core.messages import HumanMessage

def stream_nutrition_coach_text(user_text: str, thread_id: str = "nutrition-thread-1") -> None:
    config = {"configurable": {"thread_id": thread_id}}

    for event in agent.stream(
        {"messages": [HumanMessage(content=user_text)]},
        config=config,
        stream_mode="updates",
    ):
        pprint(event)
        print("-" * 80, flush=True)

In [43]:
stream_nutrition_coach_text(
    "Search the web for 3 healthy lentil and rice recipes and summarize them."
)

{'model': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 622, 'prompt_tokens': 6825, 'total_tokens': 7447, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 512, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 5760}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-DENLoqnqMwK1Kc0WrWO8nt8ksZrqw', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019ca66d-8ad8-7a82-8da0-8b8d6df7ab73-0', tool_calls=[{'name': 'web_search', 'args': {'query': 'Mediterranean lentils and rice one-pot'}, 'id': 'call_0fX3nLphWV9jctK5FoZolG4E', 'type': 'tool_call'}, {'name': 'web_search', 'args': {'query': 'One Pot Mujadara lentils and rice recipe'}, 'id': 'call_Elt0325KHE7lBJbkLzHAxO3w', 'type': 'tool_call'}, {'name': 'web_search', 